# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [84]:
import pandas as pd
import numpy as np
from scipy import stats

In [85]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [86]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [87]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [88]:
#Quesion1:What test is appropriate for this problem? Does CLT apply?

In [89]:
#we should test the requirement for CLT
#1. Independence: n<10% of populations:4870<10% population
#2. Sample Size: np>=10 ,n(1-p)>10

In [90]:
data.call.describe()

count    4870.000000
mean        0.080493
std         0.272079
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [91]:
df=data[['race','call']]
df.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [92]:
df.call.describe()

count    4870.000000
mean        0.080493
std         0.272079
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [93]:
df_b = df[df.race == 'b']
df_w=df[df.race == 'w']

In [94]:
#To evaluate these conditions, we must first calculate the pooled proportion.
#The pooled proportion is the best-estimate of the callback rate, if the null hypothesis is true.

n = len(df)
pooled = sum(df.call) / n
pooled

0.080492813141683772

In [95]:
#Since the data is from a randomized field experiment involving nearly 5,000 résumés sent in response to over 1,300 newspaper ads, the callback results can be treated as independant. To evaluate whether the success-failure condition for the normal model is met, the number of success (callbacks) and failures (no callback) from each group must be at least 10, 
#based on the pooled proportion.
## CLT applied because the sample size for each of the cases mentioned above is more than 10
#to perform a statistical analysis on whether race has a significant impact on the rate of callbacks for resumes,
#we will use the difference between two proportions. 
#The first proportion is the proportion of resumes with black-sounding names that received a call back, 
#and the second proportion is for resumes with white-sounding names that received a call back.
#The null hypothesis (H0) is that there is no difference in the two proportions. 
#The alternative hypothesis (H1) is that there is difference in the proportion of resumes that received callbacks, based on race.

In [96]:
# determine the number of black and white applicants
black = df[df.race=='b']
n_black=len(black)
white = df[df.race=='w']
n_white=len(white)
calls_b=black[black.call==1]
noCalls_b=black[black.call==0]
calls_w=white[white.call==1]
noCalls_w=white[white.call==0]
print ("Callbacks, Black", len(calls_b))
print ("No Callbacks, Black", len(noCalls_b))
print ("Callbacks, White", len(calls_w))
print ("No Callbacks, White", len(noCalls_w))

Callbacks, Black 157
No Callbacks, Black 2278
Callbacks, White 235
No Callbacks, White 2200


In [97]:
#The success-failure condition is satisfied (all values greater than 10), so the normal model applies.
##############Hypothesis Testing##################
#To test the hypothesis, first calculate a point estimate of the difference in callback rates between the two races. 
#Then calculate the standard error using the pooled proportion.
#Null hypothesis: There is no statistically significant difference in the proportion of call backs between white and black in the population
#Alternative hypothesis: There exists a statistically significant difference in the proportion of call backs between black and white in the population
p_black=len(calls_b)/n_black
p_white=len(calls_w)/n_white
p_diff=p_white-p_black
p_diff

0.032032854209445585

In [98]:
# Std Error calculated using the pooled proportion
SE = ( (pooled*(1-pooled) / n_white) + (pooled*(1-pooled) / n_black) ) ** 0.5
SE

0.0077968940361704568

In [105]:
p_null = 0
#z statistics
z = (p_diff - p_null) / SE
z

4.1084121524343464

In [104]:
p = stats.norm.sf(abs(z))*2
p
#Since the p-value is much smaller than the significance level (0.05), there is convincing evidence to reject the null hypothesis. Therefore, there was a statistically significant difference in the proportions of resumes that received callbacks between the two races examined in this study.

3.9838868375850767e-05

In [102]:
#Confidence Intervals
#For a 95% confidence interval, use a z* = 1.96. The margin of error (ME) and confidence interval (CI) are:
me=1.96*SE
print ('Margin of Error:', me)

CI = (p_diff - me, p_diff + me)
print ('Confidence Interval:', CI)


Margin of Error: 0.0152819123109
Confidence Interval: (0.016750941898551489, 0.047314766520339682)


In [ ]:
#the callback rate for job applications was 1.6 to 4.7 percent higher for white race resumes, compared to those with black race resumes.